<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/ComfyUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 ComfyUI Professional Launcher
# @markdown ### ⚙️ Configuration
USE_GOOGLE_DRIVE = False # @param {type:"boolean"}
# @markdown <font size="2" color="#888">✅ **Checked**: Saves installation & models to Google Drive (Persistent).<br>❌ **Unchecked**: Installs to Colab temporary storage (Fastest, but lost on disconnect).</font>

UPDATE_COMFY_UI = True # @param {type:"boolean"}
# @markdown <font size="2" color="#888">✅ **Checked**: Updates ComfyUI to the latest version on launch.<br>❌ **Unchecked**: Uses the existing version (Use if an update breaks your workflow).</font>

import os
import subprocess
import time
import urllib.request
import re
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

# --- 1. UI SETUP ---
header = widgets.HTML("<h2>🎨 ComfyUI Professional Launcher</h2>")

# 1. LINK AREA (TOP - Fixed Height)
# We reserve space here so the layout never shifts.
link_area = widgets.Output(layout=widgets.Layout(height='80px', width='100%', display='flex', align_items='center', justify_content='center'))

# 2. INFO BOX
info_html = widgets.HTML("""
<div style="
    padding: 12px;
    background-color: #212121;
    color: #ffffff;
    border-left: 5px solid #4caf50;
    font-family: sans-serif;
    border-radius: 4px;
    font-size: 13px;
    line-height: 1.5;
    margin-bottom: 10px;
">
    <b>🟢 Status:</b> System is initializing...<br>
    <b>👉 Action:</b> The <b>GREEN BUTTON</b> will appear above when ready.<br>
    <b>⚠️ Note:</b> Do not close this browser tab.
</div>
""")

status_label = widgets.Label(value="Initializing environment...", style={'description_width': 'initial'})
progress_bar = widgets.IntProgress(value=0, min=0, max=100, layout=widgets.Layout(width='100%'))

# 3. LOG AREA (BOTTOM - Isolated)
log_area = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        height='250px',
        overflow_y='scroll',
        border='1px solid #444',
        padding='5px'
    )
)
with log_area:
    display(widgets.HTML("<style>body { background-color: #1e1e1e; color: #d4d4d4; font-family: monospace; font-size: 12px; }</style>"))

log_acc = widgets.Accordion(children=[log_area])
log_acc.set_title(0, '📝 System Logs (Click to expand)')
log_acc.selected_index = None

# Display Order: Header -> LINK (Top) -> Info -> Status -> Progress -> LOGS (Bottom)
display(widgets.VBox([header, link_area, info_html, status_label, progress_bar, log_acc]))

# --- 2. CORE FUNCTIONS ---
ROOT = Path("/content")

def log(msg, important=False):
    """Thread-safe logging to the Output widget"""
    timestamp = time.strftime("%H:%M:%S")
    with log_area:
        print(f"[{timestamp}] {msg}")
    if important:
        status_label.value = f"👉 {msg}"

def kill_zombies():
    """Cleans up old processes"""
    log("🧹 Cleaning up background processes...")
    subprocess.run(["pkill", "-f", "cloudflared"], check=False)
    subprocess.run(["pkill", "-f", "comfyui"], check=False)
    subprocess.run(["fuser", "-k", "8188/tcp"], capture_output=True, check=False)

def run_cmd(cmd, cwd=None):
    """Runs a command and streams output"""
    process = subprocess.Popen(
        cmd, cwd=cwd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            log(output.strip())
    if process.returncode != 0:
        raise Exception(f"Command failed: {cmd}")

# --- 3. MAIN EXECUTION FLOW ---
try:
    # A. Preparation
    kill_zombies()

    # Initial Placeholder
    with link_area:
        display(widgets.HTML("<div style='color: #888; font-family: sans-serif; font-size: 14px;'>⏳ Waiting for server to start...</div>"))

    # B. Storage Logic
    if USE_GOOGLE_DRIVE:
        from google.colab import drive
        if not os.path.exists('/content/drive'):
            log("Mounting Google Drive...", important=True)
            drive.mount('/content/drive')
        INSTALL_DIR = Path("/content/drive/MyDrive/ComfyUI")
        log(f"📂 Target: Google Drive ({INSTALL_DIR})")
    else:
        INSTALL_DIR = Path("/content/ComfyUI")
        log("📂 Target: System Storage (Temporary)")

    progress_bar.value = 10

    # C. Installation
    if not INSTALL_DIR.exists():
        log("⬇️ Cloning ComfyUI Repository...", important=True)
        run_cmd(f"git clone https://github.com/comfyanonymous/ComfyUI {INSTALL_DIR}")
    elif UPDATE_COMFY_UI:
        log("🔄 Checking for ComfyUI Updates...", important=True)
        run_cmd("git pull", cwd=INSTALL_DIR)

    progress_bar.value = 30

    # D. ComfyUI Manager
    MANAGER_DIR = INSTALL_DIR / "custom_nodes" / "ComfyUI-Manager"
    if not MANAGER_DIR.exists():
        log("📦 Installing ComfyUI Manager...", important=True)
        run_cmd(f"git clone https://github.com/ltdrdata/ComfyUI-Manager {MANAGER_DIR}")

    progress_bar.value = 50

    # E. Dependencies
    log("🛠️ Verifying Python Environment...", important=True)
    run_cmd(f"pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121", cwd=INSTALL_DIR)

    progress_bar.value = 70

    # F. Cloudflare Tunnel
    CLOUDFLARED = ROOT / "cloudflared-linux-amd64"
    if CLOUDFLARED.exists():
        CLOUDFLARED.unlink()

    log("☁️ Downloading Cloudflare Tunnel...", important=True)
    run_cmd(f"wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O {CLOUDFLARED}")
    os.chmod(CLOUDFLARED, 0o777)

    progress_bar.value = 85

    # G. Launch Services
    log("⚡ Starting Background Services...", important=True)
    tunnel_log = ROOT / "cloudflared.log"
    if tunnel_log.exists(): tunnel_log.unlink()

    # 1. Start Tunnel (HTTP2)
    tunnel_cmd = f"{CLOUDFLARED} tunnel --protocol http2 --url http://127.0.0.1:8188 --metrics localhost:45678 > {tunnel_log} 2>&1 &"
    subprocess.Popen(tunnel_cmd, shell=True)

    # 2. Start ComfyUI
    comfy_cmd = f"python main.py --dont-print-server --listen --port 8188"
    comfy_proc = subprocess.Popen(
        comfy_cmd, cwd=INSTALL_DIR, shell=True,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
    )

    progress_bar.value = 95
    progress_bar.bar_style = 'warning'

    # H. Monitoring Loop
    status_label.value = "⏳ Waiting for Server & Tunnel..."
    url = None
    is_ready = False
    tunnel_retries = 0
    button_shown = False # <--- FLAG TO PREVENT GLITCHING

    while True:
        # 1. Check Process Health
        if comfy_proc.poll() is not None:
            log("❌ CRITICAL: ComfyUI Process Crashed!", important=True)
            progress_bar.bar_style = 'danger'
            break

        # 2. Check Localhost
        if not is_ready:
            try:
                with urllib.request.urlopen("http://127.0.0.1:8188", timeout=1) as response:
                    if response.getcode() == 200:
                        is_ready = True
                        log("✅ ComfyUI is listening on localhost")
            except Exception:
                pass

        # 3. Check Tunnel
        if url is None and tunnel_log.exists():
            try:
                with open(tunnel_log, "r") as f:
                    content = f.read()
                    found = re.search(r'https://[\w-]+\.trycloudflare\.com', content)
                    if found:
                        url = found.group(0)
                        log(f"🔗 Tunnel URL: {url}")

                    if "failed to parse quick Tunnel ID" in content and tunnel_retries < 3:
                        log("⚠️ Tunnel error detected. Restarting Cloudflare...", important=True)
                        subprocess.run(["pkill", "-f", "cloudflared"], check=False)
                        if tunnel_log.exists(): tunnel_log.unlink()
                        time.sleep(2)
                        subprocess.Popen(tunnel_cmd, shell=True)
                        tunnel_retries += 1
            except Exception:
                pass

        # 4. Success State (RENDER BUTTON ONLY ONCE)
        if is_ready and url and not button_shown:
            progress_bar.value = 100
            progress_bar.bar_style = 'success'
            status_label.value = "✅ System Operational"

            with link_area:
                link_area.clear_output()
                display(widgets.HTML(f"""
                <a href="{url}" target="_blank" style="
                    background-color: #28a745;
                    color: white;
                    padding: 12px 25px;
                    font-size: 16px;
                    font-weight: bold;
                    text-decoration: none;
                    border-radius: 8px;
                    font-family: sans-serif;
                    box-shadow: 0 4px 6px rgba(0,0,0,0.2);
                ">🚀 OPEN COMFYUI</a>
                """))

            button_shown = True # <--- LOCK THE BUTTON SO IT DOESN'T FLICKER

        # 5. Keep-Alive & Log Streaming
        line = comfy_proc.stdout.readline()
        if line:
            log(line.strip())

        time.sleep(0.5)

except KeyboardInterrupt:
    log("🛑 Stopped by User.")
    progress_bar.bar_style = 'danger'
    kill_zombies()

except Exception as e:
    log(f"❌ ERROR: {e}", important=True)
    progress_bar.bar_style = 'danger'